In [4]:
import pandas as pd
import dask.dataframe as dd

import bamboolib as bam
import numpy as np
import math

import matplotlib
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler

print("Done!")

Done!


## Segédfüggvények

In [5]:
def pdata_to_tdata_split(parquet_data,train_percent,val_percent):

  if train_percent + val_percent > 100:
    print("Sum of Train and Val must be less than 100\n")
    return
    
  d_parquet_len = len(parquet_data)

  first_key = parquet_data.index.start
  str_date = str(parquet_data.tpep_pickup_datetime[first_key])
  int_date = int(str_date[0:4])

  train_size = int(train_percent/100*d_parquet_len)
  val_size = int(val_percent/100*d_parquet_len)

  TrainX = torch.rand(train_size,7)
  TrainY = torch.rand(train_size,2)

  ValX = torch.rand(val_size,7)
  ValY = torch.rand(val_size,2)

  if int_date == 2019:
    valy = torch.tensor([1,0],dtype=torch.float)
  if int_date == 2020:
    valy = torch.tensor([0,1],dtype=torch.float)

  for i in range(train_size):
    TrainX[i,0:7] = torch.tensor([parquet_data.trip_distance[i],parquet_data.passenger_count[i],parquet_data.payment_type[i] + 1,
                        parquet_data.fare_amount[i],parquet_data.extra[i],parquet_data.tip_amount[i],parquet_data.improvement_surcharge[i]],dtype=torch.float)

    TrainY[i,0:2] = valy

  for i in range(val_size):
    ValX[i,0:7] = torch.tensor([parquet_data.trip_distance[i+train_size],parquet_data.passenger_count[i+train_size],parquet_data.payment_type[i+train_size] + 1,
                        parquet_data.fare_amount[i+train_size],parquet_data.extra[i+train_size],parquet_data.tip_amount[i+train_size],parquet_data.improvement_surcharge[i+train_size]],dtype=torch.float)
  
    ValY[i,0:2] = valy

  return TrainX,TrainY,  ValX,ValY


def make_dataloaders(l_parquet_datas,batch_size,train_percent=80,val_percent=20):

  whole_train_size = 0
  whole_val_size = 0

  for data in l_parquet_datas:
    d_parquet_len = len(data)
    d_train_size = int(train_percent/100*d_parquet_len)
    d_val_size = int(val_percent/100*d_parquet_len)

    whole_train_size += d_train_size
    whole_val_size += d_val_size
  
  TrainX = torch.rand(whole_train_size,7)
  TrainY = torch.rand(whole_train_size,2)

  ValX = torch.rand(whole_val_size,7)
  ValY = torch.rand(whole_val_size,2)

  i = 0
  j = 0

  for data in l_parquet_datas:

    d_parquet_len = len(data)
    d_train_size = int(train_percent/100*d_parquet_len)
    d_val_size = int(val_percent/100*d_parquet_len)

    TrainX1,TrainY1,ValX1,ValY1 = pdata_to_tdata_split(data,train_percent,val_percent)

    TrainX[i:i+d_train_size,0:7] = TrainX1
    TrainY[i:i+d_train_size,0:2] = TrainY1

    ValX[j:j+d_val_size,0:7] = ValX1
    ValY[j:j+d_val_size,0:2] = ValY1

    i += d_train_size
    j += d_val_size

  train_dataset = TensorDataset(TrainX,TrainY)
  train_dataloader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)

  val_dataset = TensorDataset(ValX,ValY)
  val_dataloader = DataLoader(val_dataset,batch_size=1,shuffle=True)

  print("Done!")
  
  return train_dataloader,val_dataloader


def make_val_dataloader(l_parquet_datas):
  whole_test_size = 0

  for parquet_data in l_parquet_datas:
    size = len(parquet_data)
    whole_test_size += size
  
  TestX = torch.rand(whole_test_size,7)
  TestY = torch.rand(whole_test_size,2)

  k = 0
  for parquet_data in l_parquet_datas:
    size = len(parquet_data)

    first_key = parquet_data.index.start
    str_date = str(parquet_data.tpep_pickup_datetime[first_key])
    int_date = int(str_date[0:4])

    if int_date == 2019:
      testy = torch.tensor([1,0],dtype=torch.float)
    if int_date == 2020:
      testy = torch.tensor([0,1],dtype=torch.float)

    TestX1 = torch.rand(size,7)
    TestY1 = torch.rand(size,2)

    j = first_key
    for i in range(size):
      TestX1[i,0:7] = torch.tensor([parquet_data.trip_distance[i+j],parquet_data.passenger_count[i+j],parquet_data.payment_type[i+j] + 1,
                        parquet_data.fare_amount[i+j],parquet_data.extra[i+j],parquet_data.tip_amount[i+j],parquet_data.improvement_surcharge[i+j]],dtype=torch.float)

      TestY1[i,0:2] = testy
    

    TestX[k:k+size,0:7] = TestX1
    TestY[k:k+size,0:2] = TestY1

    k += size

  test_dataset = TensorDataset(TestX,TestY)
  test_dataloader = DataLoader(test_dataset,batch_size=1,shuffle=True)

  print("Done!")
  
  return test_dataloader

print("Done!")

Done!


In [6]:
def train_loop(model,loss_fn,dataloader,optimizer,scheduler):

  size = len(dataloader.dataset)

  correct = 0
  running_loss = 0
  bestLoss = 1000

  for X,y in dataloader:
      optimizer.zero_grad()

      y_pred = model(X)
      correct += (y_pred.argmax() == y.argmax()).type(torch.float).sum().item()
      loss = loss_fn(y_pred,y)

      loss.backward()
      optimizer.step()

      running_loss += loss.item()
  
  scheduler.step()

  running_loss /= len(dataloader)
  correct /= size

  print(f'Loss: {running_loss}')
  if running_loss < bestLoss:
    bestLoss = running_loss
    torch.save(model.state_dict(),"myNet.pth")
    

def test_loop(model,dataloader,percent):
  size = percent/100*len(dataloader.dataset)
  correct = 0

  i = 0
  with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            correct += (pred.argmax() == y.argmax()).type(torch.float).sum().item()

            i += 1
            if i >= size:
              break

  correct /= size

  print(f"Test Accuracy: {(100*correct):>0.2f}%\n")

    
print("Done!")

Done!


In [7]:
def myOptimizers(model,lr,idx):
    m = idx % 3
    if m == 0:
        return optim.Adam(model.parameters(),lr=lr,weight_decay=1e-4)
    if m == 1:
        return optim.RMSprop(model.parameters(), lr=lr)
    if m == 2:
        return torch.optim.Rprop(model.parameters(), lr=lr)


def myLossFunctions(rnd):
    if rnd == 0:
        return nn.CrossEntropyLoss()
    if rnd == 1:
        return nn.MSELoss()

print("Done!")

Done!


## Adatok importálása

In [8]:
p2019_01_file = r'C:\Users\User\Desktop\parquet_files\yellow_tripdata_2019-01.parquet'
p2019_02_file = r'C:\Users\User\Desktop\parquet_files\yellow_tripdata_2019-02.parquet'


p2020_03_file = r'C:\Users\User\Desktop\parquet_files\yellow_tripdata_2020-03.parquet'
p2020_04_file = r'C:\Users\User\Desktop\parquet_files\yellow_tripdata_2020-04.parquet'
p2020_05_file = r'C:\Users\User\Desktop\parquet_files\yellow_tripdata_2020-05.parquet'

#jóval covid előtti
data_2019_01 = pd.read_parquet(p2019_01_file, engine='auto')
data_2019_02 = pd.read_parquet(p2019_02_file, engine='auto')

#covid pánik
data_2020_03 = pd.read_parquet(p2020_03_file, engine='auto')
data_2020_04 = pd.read_parquet(p2020_04_file, engine='auto')
data_2020_05 = pd.read_parquet(p2020_05_file, engine='auto')

print("Done!")

Done!


In [49]:
dd_ = dd.read_parquet(p2020_03_file, engine='auto')

In [50]:
dd_

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
npartitions=1,,,,,,,,,,,,,,,,,,,
,int64,datetime64[ns],datetime64[ns],float64,float64,float64,object,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [59]:
pd_

         VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0               1  2020-03-01 00:31:13   2020-03-01 01:01:42              1.0   
1               2  2020-03-01 00:08:22   2020-03-01 00:08:49              1.0   
2               1  2020-03-01 00:52:18   2020-03-01 00:59:16              1.0   
3               2  2020-03-01 00:47:53   2020-03-01 00:50:57              2.0   
4               1  2020-03-01 00:43:19   2020-03-01 00:58:27              0.0   
...           ...                  ...                   ...              ...   
3007682         2  2020-03-31 23:21:00   2020-03-31 23:33:00              NaN   
3007683         2  2020-03-31 23:57:00   2020-04-01 00:26:00              NaN   
3007684         2  2020-03-31 23:22:01   2020-03-31 23:43:52              NaN   
3007685         2  2020-03-31 23:18:53   2020-03-31 23:32:21              NaN   
3007686         2  2020-03-31 23:11:00   2020-03-31 23:30:00              NaN   

         trip_distance  Rat

In [57]:
for i in range(len(dd_)): 
    dd_['Travel_time_in_secs'][i] = (dd_['tpep_dropoff_datetime'][i]-dd_['tpep_pickup_datetime'][i]).total_seconds()

KeyError: 'Travel_time_in_secs'

In [56]:
type(pd_['tpep_dropoff_datetime'][0]-pd_['tpep_pickup_datetime'][0])

pandas._libs.tslibs.timedeltas.Timedelta

In [52]:
for i in range(10):   
    data_2020_03['Travel_time_in_secs'][i] = (data_2020_03['tpep_dropoff_datetime'][i]-data_2020_03['tpep_pickup_datetime'][i]).total_seconds()

In [46]:
data_2020_03

         VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0               1  2020-03-01 00:31:13   2020-03-01 01:01:42              1.0   
1               2  2020-03-01 00:08:22   2020-03-01 00:08:49              1.0   
2               1  2020-03-01 00:52:18   2020-03-01 00:59:16              1.0   
3               2  2020-03-01 00:47:53   2020-03-01 00:50:57              2.0   
4               1  2020-03-01 00:43:19   2020-03-01 00:58:27              0.0   
...           ...                  ...                   ...              ...   
3007682         2  2020-03-31 23:21:00   2020-03-31 23:33:00              NaN   
3007683         2  2020-03-31 23:57:00   2020-04-01 00:26:00              NaN   
3007684         2  2020-03-31 23:22:01   2020-03-31 23:43:52              NaN   
3007685         2  2020-03-31 23:18:53   2020-03-31 23:32:21              NaN   
3007686         2  2020-03-31 23:11:00   2020-03-31 23:30:00              NaN   

         trip_distance  Rat

In [37]:
import datetime

i = 0

difference = (data_2020_03['tpep_dropoff_datetime'][i]-data_2020_03['tpep_pickup_datetime'][i]).total_seconds()
print(difference)

#vendor ID: szolgáltatók
#utazás ideje mp-ben


1829.0
27.0
418.0
184.0
908.0
1114.0
1875.0
522.0
558.0
361.0
611.0
753.0
785.0
1874.0
518.0
1337.0
1011.0
396.0
573.0
446.0
377.0
201.0
811.0
1093.0
1109.0
685.0
579.0
690.0
944.0
903.0
465.0
926.0
371.0
1024.0
382.0
338.0
278.0
397.0
717.0
270.0
163.0
281.0
1884.0
1960.0
180.0
222.0
697.0
645.0
571.0
1184.0
850.0
2312.0
749.0
830.0
2232.0
396.0
903.0
979.0
1485.0
403.0
1092.0
1492.0
1091.0
909.0
808.0
1250.0
1059.0
124.0
509.0
907.0
399.0
1215.0
605.0
120.0
1118.0
1112.0
832.0
308.0
767.0
1008.0
873.0
331.0
1049.0
331.0
257.0
1337.0
1448.0
1117.0
1069.0
311.0
411.0
241.0
320.0
327.0
423.0
619.0
566.0
768.0
333.0
1903.0
367.0
298.0
908.0
609.0
745.0
1115.0
920.0
233.0
694.0
486.0
827.0
921.0
217.0
1141.0
464.0
884.0
997.0
363.0
1506.0
216.0
476.0
258.0
146.0
599.0
343.0
953.0
718.0
84113.0
860.0
352.0
396.0
536.0
399.0
748.0
404.0
358.0
526.0
907.0
512.0
296.0
393.0
636.0
222.0
612.0
318.0
1037.0
252.0
353.0
261.0
629.0
299.0
383.0
800.0
910.0
2960.0
603.0
103.0
1366.0
332.0
1445.0
13

564.0
1871.0
1492.0
628.0
200.0
378.0
410.0
246.0
294.0
518.0
1151.0
454.0
1540.0
201.0
1308.0
465.0
1445.0
273.0
387.0
506.0
410.0
644.0
763.0
914.0
179.0
440.0
309.0
172.0
131.0
850.0
154.0
613.0
270.0
105.0
313.0
607.0
410.0
610.0
143.0
511.0
947.0
1286.0
185.0
1702.0
173.0
833.0
562.0
429.0
387.0
765.0
563.0
585.0
737.0
920.0
663.0
1044.0
519.0
471.0
555.0
481.0
1344.0
920.0
545.0
2235.0
579.0
728.0
794.0
319.0
1294.0
262.0
725.0
691.0
783.0
645.0
1174.0
448.0
750.0
1766.0
980.0
1430.0
838.0
466.0
227.0
344.0
919.0
892.0
245.0
580.0
835.0
479.0
1743.0
256.0
248.0
630.0
991.0
949.0
508.0
1369.0
1206.0
204.0
674.0
768.0
231.0
327.0
1248.0
210.0
821.0
1573.0
411.0
894.0
1176.0
1165.0
376.0
450.0
577.0
272.0
335.0
1284.0
627.0
347.0
530.0
390.0
887.0
968.0
296.0
557.0
154.0
127.0
1677.0
128.0
202.0
776.0
400.0
649.0
200.0
440.0
885.0
763.0
465.0
465.0
694.0
412.0
489.0
1027.0
212.0
335.0
644.0
675.0
428.0
1462.0
1168.0
339.0
124.0
948.0
673.0
360.0
287.0
550.0
1018.0
618.0
401.0
587.0


640.0
320.0
590.0
590.0
670.0
410.0
357.0
646.0
298.0
286.0
1012.0
950.0
530.0
1419.0
1302.0
663.0
348.0
582.0
351.0
827.0
1168.0
656.0
228.0
465.0
684.0
573.0
474.0
508.0
350.0
1480.0
603.0
1027.0
984.0
465.0
606.0
623.0
100.0
739.0
1067.0
988.0
206.0
169.0
141.0
458.0
472.0
1076.0
312.0
304.0
615.0
684.0
538.0
487.0
744.0
792.0
412.0
624.0
883.0
655.0
406.0
1989.0
768.0
137.0
1140.0
553.0
1279.0
924.0
1747.0
34.0
537.0
744.0
476.0
599.0
1704.0
1339.0
1097.0
365.0
295.0
1205.0
998.0
456.0
992.0
208.0
609.0
642.0
337.0
1211.0
610.0
672.0
725.0
771.0
229.0
667.0
409.0
1319.0
1782.0
478.0
441.0
848.0
594.0
673.0
295.0
769.0
334.0
301.0
619.0
464.0
1153.0
458.0
320.0
569.0
456.0
182.0
314.0
516.0
585.0
86188.0
577.0
3028.0
283.0
515.0
283.0
605.0
225.0
330.0
217.0
580.0
909.0
290.0
1374.0
454.0
668.0
669.0
1011.0
372.0
267.0
1010.0
1739.0
236.0
482.0
482.0
429.0
360.0
971.0
576.0
948.0
634.0
369.0
1299.0
491.0
367.0
497.0
948.0
170.0
872.0
5.0
1032.0
85795.0
442.0
347.0
251.0
646.0
625.0


676.0
2336.0
1750.0
929.0
654.0
398.0
355.0
583.0
1092.0
236.0
881.0
1033.0
1499.0
819.0
1058.0
580.0
791.0
1509.0
1909.0
533.0
774.0
1615.0
1082.0
914.0
1588.0
267.0
1012.0
1775.0
340.0
1872.0
478.0
1074.0
358.0
1285.0
227.0
863.0
605.0
514.0
184.0
1091.0
452.0
327.0
133.0
590.0
623.0
478.0
479.0
1287.0
889.0
502.0
280.0
323.0
1349.0
565.0
307.0
1132.0
167.0
327.0
420.0
631.0
363.0
332.0
258.0
814.0
453.0
860.0
830.0
796.0
723.0
361.0
662.0
1695.0
653.0
192.0
257.0
1488.0
298.0
345.0
1050.0
1928.0
1225.0
379.0
597.0
1686.0
544.0
782.0
875.0
1749.0
721.0
618.0
153.0
1519.0
298.0
302.0
178.0
543.0
683.0
624.0
1343.0
207.0
919.0
674.0
1516.0
375.0
646.0
1367.0
599.0
1310.0
1737.0
334.0
678.0
1340.0
335.0
608.0
177.0
202.0
474.0
1674.0
400.0
648.0
967.0
912.0
1206.0
2405.0
565.0
441.0
366.0
513.0
396.0
793.0
1946.0
1113.0
600.0
316.0
757.0
793.0
776.0
928.0
458.0
824.0
654.0
299.0
517.0
334.0
267.0
2987.0
446.0
116.0
542.0
202.0
202.0
445.0
329.0
803.0
421.0
445.0
258.0
413.0
1008.0
1791.

305.0
1352.0
2215.0
185.0
1390.0
186.0
450.0
407.0
177.0
344.0
229.0
1149.0
434.0
1251.0
852.0
433.0
2318.0
574.0
678.0
169.0
794.0
413.0
869.0
974.0
316.0
1415.0
795.0
260.0
376.0
647.0
200.0
716.0
313.0
912.0
529.0
33.0
470.0
403.0
891.0
1833.0
829.0
566.0
627.0
1476.0
134.0
602.0
302.0
367.0
901.0
207.0
254.0
2117.0
470.0
451.0
564.0
495.0
421.0
273.0
367.0
591.0
791.0
595.0
983.0
1029.0
748.0
373.0
1802.0
563.0
662.0
3270.0
332.0
554.0
554.0
999.0
405.0
1245.0
818.0
1035.0
232.0
840.0
970.0
812.0
309.0
232.0
1103.0
879.0
1572.0
526.0
301.0
1370.0
1129.0
392.0
1063.0
293.0
985.0
2500.0
969.0
86175.0
898.0
434.0
970.0
663.0
825.0
263.0
133.0
639.0
121.0
519.0
641.0
425.0
899.0
789.0
950.0
728.0
184.0
547.0
1102.0
736.0
1137.0
492.0
617.0
1058.0
450.0
367.0
538.0
1486.0
378.0
596.0
379.0
191.0
589.0
755.0
292.0
542.0
215.0
343.0
453.0
854.0
435.0
683.0
595.0
818.0
411.0
550.0
969.0
1193.0
304.0
1743.0
252.0
532.0
331.0
953.0
726.0
394.0
1850.0
287.0
983.0
750.0
875.0
351.0
347.0
766.0

...

208.0
464.0
347.0
371.0
741.0
418.0
1049.0
589.0
1545.0
451.0
232.0
243.0
230.0
233.0
503.0
419.0
870.0
196.0
196.0
361.0
469.0
1283.0
602.0
841.0
718.0
1111.0
802.0
0.0
719.0
966.0
524.0
2115.0
226.0
993.0
435.0
577.0
561.0
814.0
1113.0
112.0
999.0
552.0
557.0
1286.0
470.0
1003.0
1388.0
1907.0
775.0
222.0
690.0
759.0
702.0
1070.0
402.0
947.0
282.0
567.0
311.0
1045.0
793.0
369.0
342.0
1172.0
302.0
712.0
375.0
541.0
667.0
173.0
1177.0
846.0
538.0
453.0
795.0
893.0
639.0
191.0
212.0
764.0
296.0
654.0
586.0
49.0
49.0
400.0
784.0
934.0
523.0
704.0
1099.0
183.0
362.0
844.0
1145.0
122.0
951.0
1085.0
435.0
965.0
336.0
1755.0
350.0
936.0
1359.0
1057.0
1852.0
148.0
525.0
491.0
474.0
966.0
551.0
306.0
886.0
291.0
1441.0
1105.0
411.0
746.0
698.0
1571.0
422.0
2415.0
903.0
473.0
581.0
421.0
1017.0
380.0
1505.0
182.0
362.0
18.0
956.0
323.0
486.0
386.0
371.0
523.0
427.0
1034.0
1667.0
589.0
877.0
591.0
592.0
572.0
414.0
308.0
309.0
164.0
371.0
295.0
169.0
389.0
616.0
1514.0
124.0
401.0
341.0
426.0
800

682.0
216.0
271.0
1682.0
447.0
778.0
778.0
1055.0
496.0
667.0
330.0
630.0
2234.0
561.0
608.0
1485.0
488.0
505.0
352.0
463.0
964.0
1284.0
329.0
443.0
766.0
587.0
344.0
321.0
295.0
142.0
1422.0
16.0
2725.0
1164.0
1253.0
735.0
384.0
644.0
705.0
490.0
466.0
572.0
234.0
629.0
355.0
1044.0
1580.0
407.0
222.0
2042.0
1713.0
643.0
1459.0
1111.0
1924.0
2012.0
1382.0
516.0
498.0
1241.0
2253.0
1038.0
373.0
364.0
350.0
106.0
2289.0
615.0
463.0
1054.0
801.0
503.0
592.0
888.0
716.0
398.0
949.0
393.0
484.0
1834.0
892.0
184.0
3060.0
1586.0
1735.0
962.0
649.0
673.0
418.0
2882.0
551.0
215.0
215.0
272.0
359.0
2280.0
359.0
640.0
640.0
629.0
1960.0
317.0
482.0
1126.0
327.0
379.0
932.0
1001.0
3023.0
1476.0
250.0
153.0
767.0
323.0
352.0
1242.0
1464.0
605.0
2104.0
596.0
737.0
759.0
254.0
732.0
654.0
249.0
242.0
2498.0
1602.0
483.0
2145.0
400.0
0.0
674.0
487.0
198.0
410.0
456.0
311.0
952.0
715.0
400.0
744.0
1093.0
1699.0
1122.0
843.0
1118.0
1183.0
469.0
397.0
1781.0
935.0
827.0
504.0
906.0
533.0
489.0
2.0
686.0

886.0
607.0
981.0
958.0
1144.0
428.0
646.0
649.0
1242.0
1051.0
283.0
390.0
764.0
380.0
1867.0
868.0
329.0
1758.0
560.0
785.0
739.0
375.0
526.0
957.0
485.0
216.0
164.0
222.0
599.0
364.0
315.0
1414.0
633.0
921.0
280.0
465.0
436.0
243.0
171.0
386.0
958.0
1236.0
415.0
599.0
1115.0
157.0
856.0
525.0
650.0
869.0
1049.0
665.0
469.0
285.0
606.0
2308.0
240.0
574.0
264.0
734.0
538.0
1067.0
601.0
162.0
214.0
608.0
493.0
1473.0
1367.0
346.0
1711.0
612.0
1094.0
441.0
771.0
50.0
1588.0
778.0
242.0
210.0
317.0
390.0
566.0
370.0
640.0
551.0
289.0
275.0
1055.0
466.0
538.0
1437.0
626.0
638.0
347.0
132.0
355.0
253.0
646.0
79.0
868.0
2316.0
201.0
440.0
1190.0
282.0
319.0
305.0
1274.0
1407.0
845.0
1134.0
128.0
171.0
1069.0
240.0
201.0
1024.0
2015.0
271.0
1227.0
1696.0
483.0
1021.0
277.0
618.0
2835.0
662.0
1214.0
754.0
709.0
665.0
963.0
1154.0
1200.0
240.0
206.0
1112.0
240.0
169.0
270.0
461.0
2394.0
520.0
3196.0
2342.0
861.0
956.0
354.0
579.0
532.0
296.0
1212.0
1212.0
1006.0
1113.0
510.0
170.0
211.0
262.0
8

1070.0
2593.0
241.0
666.0
353.0
1355.0
251.0
345.0
3409.0
104.0
218.0
648.0
711.0
2116.0
35485.0
291.0
812.0
2374.0
296.0
1177.0
1634.0
783.0
819.0
1444.0
1054.0
337.0
1149.0
1476.0
555.0
112.0
632.0
351.0
882.0
383.0
759.0
412.0
730.0
231.0
393.0
809.0
628.0
545.0
1170.0
782.0
117.0
1562.0
173.0
2398.0
2401.0
1693.0
1272.0
400.0
1336.0
374.0
1212.0
540.0
361.0
572.0
1427.0
225.0
359.0
1516.0
705.0
136.0
900.0
608.0
289.0
1657.0
135.0
640.0
415.0
386.0
654.0
3441.0
92.0
131.0
849.0
853.0
2849.0
2928.0
1208.0
994.0
486.0
813.0
926.0
781.0
1167.0
294.0
1024.0
669.0
757.0
647.0
802.0
360.0
196.0
822.0
225.0
498.0
1448.0
897.0
376.0
944.0
969.0
979.0
753.0
688.0
401.0
774.0
1693.0
1516.0
1130.0
516.0
567.0
475.0
75.0
390.0
2288.0
1293.0
669.0
854.0
549.0
2993.0
434.0
289.0
747.0
827.0
662.0
2478.0
456.0
192.0
375.0
2591.0
454.0
664.0
164.0
455.0
857.0
1756.0
497.0
1257.0
542.0
1832.0
234.0
1131.0
838.0
632.0
1034.0
776.0
160.0
888.0
602.0
536.0
2627.0
248.0
355.0
711.0
852.0
1304.0
280.0
6

KeyboardInterrupt: 

In [25]:
data_2020_03

         VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0               1  2020-03-01 00:31:13   2020-03-01 01:01:42              1.0   
1               2  2020-03-01 00:08:22   2020-03-01 00:08:49              1.0   
2               1  2020-03-01 00:52:18   2020-03-01 00:59:16              1.0   
3               2  2020-03-01 00:47:53   2020-03-01 00:50:57              2.0   
4               1  2020-03-01 00:43:19   2020-03-01 00:58:27              0.0   
...           ...                  ...                   ...              ...   
3007682         2  2020-03-31 23:21:00   2020-03-31 23:33:00              NaN   
3007683         2  2020-03-31 23:57:00   2020-04-01 00:26:00              NaN   
3007684         2  2020-03-31 23:22:01   2020-03-31 23:43:52              NaN   
3007685         2  2020-03-31 23:18:53   2020-03-31 23:32:21              NaN   
3007686         2  2020-03-31 23:11:00   2020-03-31 23:30:00              NaN   

         trip_distance  Rat

In [27]:
i = 2

difference = data_2020_03['tpep_dropoff_datetime'][i]-data_2020_03['tpep_pickup_datetime'][i]

seconds_in_day = 24 * 60 * 60

divmod(difference.days * seconds_in_day + difference.seconds, 60)

(6, 58)

In [9]:
data_2020 = [data_2020_03, data_2020_04]
s = 0
for d in data_2020:
  s += len(d)

data_2019 = data_2019_01[0:s]
whole_data = [data_2019]
whole_data.extend(data_2020)

print(f'Size of 2019 dataset: {len(whole_data[0])}, size of 2020 dataset: {s}')
print("Done!")

Size of 2019 dataset: 3245760, size of 2020 dataset: 3245760
Done!


## Adatok betöltése

In [7]:
batch_size = 32
train_percent = 65
test_percent = 25

print("Train and test dataloading started...")
train_dataloader, test_dataloader = make_dataloaders(whole_data,batch_size,train_percent,test_percent)

train_size = len(train_dataloader.dataset)
test_size = len(test_dataloader.dataset)

print(f'Size of train dataset: {train_size} ,size of test dataset: {test_size} proportion goodness is {(train_percent/test_percent)/(train_size/test_size)}')

Train and test dataloading started...
Done!
Size of train dataset: 4219487 ,size of test dataset: 1622879 proportion goodness is 0.9999996208069843


In [8]:
"""
val_data_2020 = [data_2020_03[int((test_percent + 1)/100*len(data_2020_03)):int((test_percent + 3)/100*len(data_2020_03))],
                 data_2020_04[int((test_percent + 1)/100*len(data_2020_04)):int((test_percent + 3)/100*len(data_2020_04))],
                 data_2020_05[int((test_percent + 1)/100*len(data_2020_05)):int((test_percent + 3)/100*len(data_2020_05))]]
"""

val_data_2020 = [data_2020_05]
Sum = 0
for d in val_data_2020:
  Sum += len(d)

#val_data_2019 = data_2019_01[int((test_percent + 1)/100*len(data_2019_01)):s+int((test_percent + 1.5)/100*len(data_2019_01))]

val_data_2019 = data_2019_02[0:Sum]
val_data = [val_data_2019]
val_data.extend(val_data_2020)

val_size_2019 = len(val_data_2019)
val_size_2020 = Sum

print(f'Size of 2019 dataset: {val_size_2019} ,size of 2020 dataset: {val_size_2020}')

Size of 2019 dataset: 348415 ,size of 2020 dataset: 348415


In [9]:
print("Val dataloading started...")
val_dataloader = make_val_dataloader(val_data)
print(f'Size of val dataset: {len(val_dataloader.dataset)}')

Val dataloading started...
Done!
Size of val dataset: 696830


## Modell osztályok

In [10]:
learning_rate = 0.01

input_size = train_dataloader.dataset.tensors[0].size(1)
output_size = train_dataloader.dataset.tensors[1].size(1)

class myLNN1(nn.Module):
  def __init__(self,Nin,Nout):
    super(myLNN1,self).__init__()

    self.L1 = nn.Linear(Nin,128)
    self.L2 = nn.Linear(128,64)
    self.L3 = nn.Linear(64,Nout)
  
  def forward(self,x):

    x = torch.relu(self.L1(x))
    x = torch.relu(self.L2(x))
    x = self.L3(x)

    return x

model1 = myLNN1(input_size,output_size)

class myLNN2(nn.Module):
  def __init__(self,Nin,Nout):
    super(myLNN2,self).__init__()

    self.L1 = nn.Linear(Nin,50)
    self.L2 = nn.Linear(50,Nout)
  
  def forward(self,x):

    x = torch.relu(self.L1(x))
    x = self.L2(x)

    return x

model2 = myLNN2(input_size,output_size)

class myLNN3(nn.Module):
  def __init__(self,Nin,Nout):
    super(myLNN3,self).__init__()

    self.L1 = nn.Linear(Nin,20)
    self.L2 = nn.Linear(20,20)
    self.L3 = nn.Linear(20,Nout)
  
  def forward(self,x):

    x = torch.relu(self.L1(x))
    x = torch.relu(self.L2(x))
    x = self.L3(x)

    return x

model3 = myLNN3(input_size,output_size)

class myLNN4(nn.Module):
  def __init__(self,Nin,Nout):
    super(myLNN4,self).__init__()

    self.L1 = nn.Linear(Nin,128)
    self.L2 = nn.Linear(128,240)
    self.L3 = nn.Linear(240,Nout)
  
  def forward(self,x):

    x = torch.relu(self.L1(x))
    x = torch.relu(self.L2(x))
    x = self.L3(x)

    return x

model4 = myLNN4(input_size,output_size)

class myLNN5(nn.Module):
  def __init__(self,Nin,Nout):
    super(myLNN5,self).__init__()

    self.L1 = nn.Linear(Nin,200)
    self.L2 = nn.Linear(200,Nout)
  
  def forward(self,x):

    x = torch.relu(self.L1(x))
    x = self.L2(x)

    return x

model5 = myLNN5(input_size,output_size)

class expert_system(nn.Module):
  def __init__(self,models,output_size):
    super(expert_system,self).__init__()

    self.model_parts = models
  
  def forward(self,x):

    outs = torch.zeros(x.size(0),output_size)
    outs = outs.clone().detach().requires_grad_(False)
    for i in range(len(self.model_parts)):
      
      outs += self.model_parts[i](x)

    return outs

In [11]:
models = [model1,model2,model3,model4,model5]
print("Done!")

Done!


## Tanítás és validálás (Single model)

In [12]:
epochNum = 3
loss_fn = nn.CrossEntropyLoss()

model = model1

optimizer = myOptimizers(model,learning_rate,1)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, epochNum, 1e-4)
for i in range(epochNum):
    print(f'Epoch {i+1}')
    train_loop(model,loss_fn,train_dataloader,optimizer,scheduler)
    test_loop(model,test_dataloader, 100)

print("Validation...")
test_loop(model,val_dataloader,100)
print("Done!")

Epoch 1
Loss: 0.5106555773896744
Test Accuracy: 75.38%

Epoch 2
Loss: 0.5041612591647431
Test Accuracy: 74.71%

Epoch 3
Loss: 0.48486805261392874
Test Accuracy: 78.66%

Validation...
Test Accuracy: 69.21%

Done!


## Tanítás és validálás (Expert system)

In [13]:
epochNum = 5

j = 0
for model in models:
    loss_fn = myLossFunctions(0)
    optimizer = myOptimizers(model,learning_rate,0)
    scheduler = lr_scheduler.CosineAnnealingLR(optimizer, epochNum, 1e-4)
    j += 1
    print(f'Model {j}')
    for i in range(epochNum):
        print(f'Epoch {i+1}')
        train_loop(model,loss_fn,train_dataloader,optimizer,scheduler)
        test_loop(model,test_dataloader, 100)

print("Done!")

Model 1
Epoch 1
Loss: 0.45888390626834136
Test Accuracy: 77.97%

Epoch 2
Loss: 0.46957412115152963
Test Accuracy: 75.86%

Epoch 3
Loss: 0.4384750187971537
Test Accuracy: 78.32%

Epoch 4
Loss: 0.4032563197975881
Test Accuracy: 79.00%

Epoch 5
Loss: 0.38532977455040485
Test Accuracy: 79.68%

Model 2
Epoch 1
Loss: 7778660.6448023645
Test Accuracy: 49.33%

Epoch 2
Loss: 3106407.702503777
Test Accuracy: 44.63%

Epoch 3
Loss: 3959201297.9434123
Test Accuracy: 52.70%

Epoch 4
Loss: 5820381001.540264
Test Accuracy: 54.55%

Epoch 5
Loss: 875591172.3865701
Test Accuracy: 49.54%

Model 3
Epoch 1
Loss: 0.504690568123516
Test Accuracy: 75.42%

Epoch 2
Loss: 0.4859419816510639
Test Accuracy: 75.89%

Epoch 3
Loss: 0.4528966846578232
Test Accuracy: 76.95%

Epoch 4
Loss: 0.41026790537789526
Test Accuracy: 78.83%

Epoch 5
Loss: 0.38170924536072826
Test Accuracy: 80.19%

Model 4
Epoch 1
Loss: 411586.32516759686
Test Accuracy: 64.36%

Epoch 2
Loss: 50257.58377240704
Test Accuracy: 50.00%

Epoch 3
Loss: 17

In [16]:
models.pop(3)

myLNN4(
  (L1): Linear(in_features=7, out_features=128, bias=True)
  (L2): Linear(in_features=128, out_features=240, bias=True)
  (L3): Linear(in_features=240, out_features=2, bias=True)
)

In [18]:
models.pop(1)

myLNN2(
  (L1): Linear(in_features=7, out_features=512, bias=True)
  (L2): Linear(in_features=512, out_features=2, bias=True)
)

In [19]:
print("Validation...")
exp_sys = expert_system(models,output_size)
test_loop(model1,val_dataloader,100)

Validation...
Test Accuracy: 69.35%



In [15]:
#torch.save(model.state_dict(), 'myExpertNet.pth')

## Vizualizáció

In [ ]:
import pandas as pd
import dask.dataframe as dd
df = dd.read_parquet('nyc_taxi_wide.parq')
df.columns
df.head()

In [ ]:
usecols = ['dropoff_x','dropoff_y','pickup_longitude','pickup_y','dropoff_hour','pickup_hour','passenger_count']
df = dd.read_parquet('yellow_tripdata_202004.parquet')[usecols].persist()

#df = dd.read_parquet('yellow_tripdata_202004.parquet', engine='pyarrow')
df.head()

In [ ]:
import hvplot.pandas
import holoviews as hv, colorcet as cc
from holoviews.element.tiles import EsriImagery
from holoviews.operation.datashader import datashade
hv.extension('bokeh')

In [ ]:
map_tiles = EsriImagery().opts(alpha=0.5, width=900, height=480, bgcolor='black')
points = hv.Points(df, ['dropoff_x', 'dropoff_y'])
taxi_trips = datashade(points, cmap=cc.fire, width=900, height=480)
map_tiles * taxi_trips

In [ ]:
map_tiles = EsriImagery().opts(alpha=0.5, width=900, height=480, bgcolor='black')
plot = df.hvplot(
    'dropoff_x',
    'dropoff_y',
    kind='scatter',
    rasterize=True,
    cmap=cc.fire,
    cnorm='eq_hist'
)
map_tiles * plot

In [ ]:
pip install azureml-opendatasets --user

In [ ]:
##új data kipróbálás


from azureml.opendatasets import NycTlcYellow

from datetime import datetime
from dateutil import parser

start_date = parser.parse('2021-01-01')
end_date = parser.parse('2022-01-01')
nyc_tlc = NycTlcYellow(start_date=start_date, end_date=end_date)
nyc_tlc_df = nyc_tlc.to_pandas_dataframe()

nyc_tlc_df.info()